# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [2]:
data = pd.read_csv("https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv",
                   sep = "~")
# Python starts the index at 0, so 14ths row is row number 13 in the data frame
data.iloc[[13, 27, 101],]

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,"Finalwt_worksite,,,,"
13,3,1.0,2.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,1437.0,4.0,6.0,7.0,3.0,0.0,"47.793940929,,,,"
27,1,3.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,1.0,2501.0,2.0,4.0,7.0,8.0,0.0,"47.793940929,,,,"
101,2,1.0,1.0,3.0,2.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,2.0,12636.0,4.0,6.0,7.0,4.0,0.0,"47.793940929,,,,"


In [3]:
data.Industry.describe()

count    2842.000000
mean        3.804715
std         1.959338
min         1.000000
25%         2.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Industry, dtype: float64

## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [4]:
mycols = ['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6'] + \
         list(data.columns[data.columns.str.startswith('WD')])
data_wv = data[mycols]
data_wv.head()

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL6,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [5]:
# data.column
data_wv = data_wv.rename({'Industry': 'Industry',
                          'Size':     'Employee_Size',
                          'OC3':      'Organization_type',
                          'HI1':      'General_Health_Insurance',
                          'HI2':      'Health_Insurance_Proportion',
                          'HI3':      'Health_Insurance_Parttime',
                          'CP1':      'Promoting_Health_at_Work',
                          'WL6':      'WorkLife_Programming_Policies_Benefits',
                          'WD1_1':    'Workforce_Demographics_Pct_Under_30',
                          'WD1_2':    'Workforce_Demographics_Pct_Over_60',
                          'WD2':      'Workforce_Demographics_Pct_Female',
                          'WD3':      'Workforce_Demographics_Pct_Hourly',
                          'WD4':      'Workforce_Demographics_Pct_Other',
                          'WD5':      'Workforce_Demographics_Pct_Remote',
                          'WD6':      'Workforce_Demographics_Pct_Union',
                          'WD7':      'Workforce_Demographics_Pct_Turnover'},
                         axis=1)
data_wv.head()

,Industry,Employee_Size,Organization_type,General_Health_Insurance,Health_Insurance_Proportion,Health_Insurance_Parttime,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce_Demographics_Pct_Under_30,Workforce_Demographics_Pct_Over_60,Workforce_Demographics_Pct_Female,Workforce_Demographics_Pct_Hourly,Workforce_Demographics_Pct_Other,Workforce_Demographics_Pct_Remote,Workforce_Demographics_Pct_Union,Workforce_Demographics_Pct_Turnover
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [6]:
replace_map = {1:'Agriculture, Forestry, Fishing and Hunting; Mining; Utilities; Construction; Manufacturing', 
               2:'Wholesale Trade; Retail Trade; Transportation and Warehousing', 
               3:'Arts, Entertainment, and Recreation; Accommodation and Food Services; Other Services (except Public Administration)', 
               4:'Information; Finance and Insurance; Real Estate Rental and Leasing; Professional, Scientific, and Technical Services; Management of Companies and Enterprises; Administrative and Support and Waste Management and Remediation Services',
               5:'Educational Services; Health Care and Social Assistance',
               6:'Public Administration',
               7:'Hospital',}
data_wv.Industry = data_wv.Industry.map(replace_map)
data_wv.Industry

0                    Hospital
1                    Hospital
2                    Hospital
3                    Hospital
4                    Hospital
                ...          
2838    Public Administration
2839    Public Administration
2840    Public Administration
2841    Public Administration
2842    Public Administration
Name: Industry, Length: 2843, dtype: object

## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [7]:
data_wv['Employee_Size'] = pd.cut(data_wv.Employee_Size, 
                                        bins=[0,3,5,8], 
                                        labels=("Small", "Medium", "Large"))
data_wv['Employee_Size'].value_counts()

Small     2195
Medium     393
Large      254
Name: Employee_Size, dtype: int64

## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [8]:
# Add more labels to the
# not going to handle WD columns
replace_map = {'Organization_type':{1:'Human Resources and/or Benefits', 
                                    2:'Health Promotion/Fitness/Wellness', 
                                    3:'Safety', 
                                    4:'Medical',
                                    5:'PR/Marketing',
                                    6:'Finance',
                                    7:'Other',
                                    97:np.nan,
                                    99:np.nan}, 
               'General_Health_Insurance':{1:'Full insurance coverage offered',
                                           2:'Partial insurance coverage offered', 
                                           3:'No insurance coverage offered', 
                                           97:np.nan,
                                           98:np.nan,
                                           99:np.nan},
               'Health_Insurance_Proportion':{1:'Larger',
                                              2:'Smaller', 
                                              3:'About the same', 
                                              96:np.nan,
                                              97:np.nan,
                                              98:np.nan,
                                              99:np.nan},
               'Health_Insurance_Parttime':{1:'Yes',
                                            2:'No', 
                                            97:np.nan,
                                            98:np.nan,
                                            99:np.nan},
               'Promoting_Health_at_Work':{1:'Yes',
                                           2:'No', 
                                           97:np.nan,
                                           98:np.nan},
               'WorkLife_Programming_Policies_Benefits':{1:'Yes',
                                                         2:'No', 
                                                         97:np.nan,
                                                         98:np.nan,
                                                         99:np.nan}}
data_wv = data_wv.replace(replace_map)
data_wv.head()

,Industry,Employee_Size,Organization_type,General_Health_Insurance,Health_Insurance_Proportion,Health_Insurance_Parttime,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce_Demographics_Pct_Under_30,Workforce_Demographics_Pct_Over_60,Workforce_Demographics_Pct_Female,Workforce_Demographics_Pct_Hourly,Workforce_Demographics_Pct_Other,Workforce_Demographics_Pct_Remote,Workforce_Demographics_Pct_Union,Workforce_Demographics_Pct_Turnover
0,Hospital,Large,Safety,Partial insurance coverage offered,Larger,No,Yes,Yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospital,Large,Safety,Partial insurance coverage offered,About the same,Yes,Yes,Yes,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,Hospital,Large,Safety,Full insurance coverage offered,About the same,Yes,Yes,Yes,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,Hospital,Medium,Health Promotion/Fitness/Wellness,Full insurance coverage offered,Smaller,Yes,No,No,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,Hospital,Medium,Safety,Full insurance coverage offered,About the same,Yes,Yes,Yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [9]:
wdcols = [x for x in data_wv.columns if x.startswith("Workforce")]
data_wv[wdcols] = data_wv[wdcols].replace([997, 998, 999], np.nan)
data_wv.head()

,Industry,Employee_Size,Organization_type,General_Health_Insurance,Health_Insurance_Proportion,Health_Insurance_Parttime,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce_Demographics_Pct_Under_30,Workforce_Demographics_Pct_Over_60,Workforce_Demographics_Pct_Female,Workforce_Demographics_Pct_Hourly,Workforce_Demographics_Pct_Other,Workforce_Demographics_Pct_Remote,Workforce_Demographics_Pct_Union,Workforce_Demographics_Pct_Turnover
0,Hospital,Large,Safety,Partial insurance coverage offered,Larger,No,Yes,Yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospital,Large,Safety,Partial insurance coverage offered,About the same,Yes,Yes,Yes,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN
2,Hospital,Large,Safety,Full insurance coverage offered,About the same,Yes,Yes,Yes,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN
3,Hospital,Medium,Health Promotion/Fitness/Wellness,Full insurance coverage offered,Smaller,Yes,No,No,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN
4,Hospital,Medium,Safety,Full insurance coverage offered,About the same,Yes,Yes,Yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [10]:
catcolumns = ['Industry', 'Employee_Size']
data_wv[catcolumns] = data_wv[catcolumns].astype('object')
data_wv.sort_values(by=['Industry', 'Employee_Size', 'Workforce_Demographics_Pct_Under_30'], 
                    ascending = [True, True, False]).head()

,Industry,Employee_Size,Organization_type,General_Health_Insurance,Health_Insurance_Proportion,Health_Insurance_Parttime,Promoting_Health_at_Work,WorkLife_Programming_Policies_Benefits,Workforce_Demographics_Pct_Under_30,Workforce_Demographics_Pct_Over_60,Workforce_Demographics_Pct_Female,Workforce_Demographics_Pct_Hourly,Workforce_Demographics_Pct_Other,Workforce_Demographics_Pct_Remote,Workforce_Demographics_Pct_Union,Workforce_Demographics_Pct_Turnover
1732,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,Health Promotion/Fitness/Wellness,Partial insurance coverage offered,About the same,No,Yes,No,50.0,10.0,50.0,75.0,10.0,0.0,0.0,75.0
1476,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,Health Promotion/Fitness/Wellness,Partial insurance coverage offered,About the same,No,Yes,No,40.0,10.0,30.0,60.0,30.0,5.0,0.0,10.0
1477,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,Health Promotion/Fitness/Wellness,Partial insurance coverage offered,Smaller,No,Yes,Yes,25.0,15.0,20.0,60.0,10.0,2.0,60.0,5.0
704,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,Health Promotion/Fitness/Wellness,Full insurance coverage offered,About the same,No,Yes,Yes,20.0,15.0,17.0,62.0,10.0,5.0,0.0,11.0
1241,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,Health Promotion/Fitness/Wellness,Full insurance coverage offered,About the same,No,Yes,Yes,20.0,25.0,50.0,70.0,20.0,5.0,0.0,3.0


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [11]:
# Before dropping the row:
data_wv.shape

(2843, 16)

In [12]:
data_wv = data_wv[data_wv['Industry'].notna()]
data_wv.shape

(2842, 16)

## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [13]:
data_wv['gender_balance'] = pd.cut(data_wv.Workforce_Demographics_Pct_Female, 
                                   bins=[0,35,65,100], 
                                   labels=("Mostly men", "Balanced", "Mostly women"))
data_wv['gender_balance'].value_counts()

Mostly women    727
Mostly men      710
Balanced        587
Name: gender_balance, dtype: int64

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

In [14]:
# Before change:
data_wv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 2842
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   Industry                                2842 non-null   object  
 1   Employee_Size                           2842 non-null   object  
 2   Organization_type                       2615 non-null   object  
 3   General_Health_Insurance                2763 non-null   object  
 4   Health_Insurance_Proportion             2249 non-null   object  
 5   Health_Insurance_Parttime               2682 non-null   object  
 6   Promoting_Health_at_Work                2668 non-null   object  
 7   WorkLife_Programming_Policies_Benefits  2549 non-null   object  
 8   Workforce_Demographics_Pct_Under_30     1874 non-null   float64 
 9   Workforce_Demographics_Pct_Over_60      1901 non-null   float64 
 10  Workforce_Demographics_Pct_Female       2047 non

In [15]:
catcolumns = ['Industry', 'Employee_Size', 'General_Health_Insurance', 
              'Organization_type', 'Health_Insurance_Proportion',
              'Health_Insurance_Parttime', 'Promoting_Health_at_Work', 
              'WorkLife_Programming_Policies_Benefits']
data_wv[catcolumns] = data_wv[catcolumns].astype('category')
data_wv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 2842
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   Industry                                2842 non-null   category
 1   Employee_Size                           2842 non-null   category
 2   Organization_type                       2615 non-null   category
 3   General_Health_Insurance                2763 non-null   category
 4   Health_Insurance_Proportion             2249 non-null   category
 5   Health_Insurance_Parttime               2682 non-null   category
 6   Promoting_Health_at_Work                2668 non-null   category
 7   WorkLife_Programming_Policies_Benefits  2549 non-null   category
 8   Workforce_Demographics_Pct_Under_30     1874 non-null   float64 
 9   Workforce_Demographics_Pct_Over_60      1901 non-null   float64 
 10  Workforce_Demographics_Pct_Female       2047 non

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

In [16]:
data_wv.query("WorkLife_Programming_Policies_Benefits == 'Yes' and Employee_Size == 'Small'").\
    stb.freq(['General_Health_Insurance'])

,General_Health_Insurance,count,percent,cumulative_count,cumulative_percent
0,Full insurance coverage offered,324,46.285714,324,46.285714
1,Partial insurance coverage offered,310,44.285714,634,90.571429
2,No insurance coverage offered,66,9.428571,700,100.000000


## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

In [17]:
# Initialize a new database
WHA_clean_db = sqlite3.connect("WHA_clean.db") 
data_wv.to_sql('data_wv', WHA_clean_db, index=False, chunksize=1000, if_exists='replace')

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

In [18]:
# SQL Version:
tmp = pd.read_sql_query("SELECT Employee_Size, Organization_type, General_Health_Insurance AS insurance, Workforce_Demographics_Pct_Female FROM data_wv \
                        WHERE Industry = 'Hospital' AND Health_Insurance_Proportion='Smaller' \
                        ORDER BY Workforce_Demographics_Pct_Female DESC", WHA_clean_db)
tmp

,Employee_Size,Organization_type,insurance,Workforce_Demographics_Pct_Female
0,Medium,Safety,Full insurance coverage offered,89.0
1,Large,Safety,Partial insurance coverage offered,80.0
2,Large,Safety,Partial insurance coverage offered,80.0
3,Small,Safety,Full insurance coverage offered,75.0
4,Medium,Safety,Partial insurance coverage offered,65.0
5,Medium,Health Promotion/Fitness/Wellness,Full insurance coverage offered,50.0
6,Medium,None,Partial insurance coverage offered,NaN
7,Medium,Safety,Partial insurance coverage offered,NaN
8,Medium,Safety,Full insurance coverage offered,NaN
9,Medium,Safety,Full insurance coverage offered,NaN


In [19]:
# Pandas Version
data_wv.query("Industry == 'Hospital' and Health_Insurance_Proportion == 'Smaller'")\
    [['Employee_Size', 'Organization_type', 
      'General_Health_Insurance', 'Workforce_Demographics_Pct_Female']].\
    rename(columns={'General_Health_Insurance': 'insurance'}).\
    sort_values(by = 'Workforce_Demographics_Pct_Female', ascending=False)

,Employee_Size,Organization_type,insurance,Workforce_Demographics_Pct_Female
320,Medium,Safety,Full insurance coverage offered,89.0
187,Large,Safety,Partial insurance coverage offered,80.0
214,Large,Safety,Partial insurance coverage offered,80.0
229,Small,Safety,Full insurance coverage offered,75.0
191,Medium,Safety,Partial insurance coverage offered,65.0
3,Medium,Health Promotion/Fitness/Wellness,Full insurance coverage offered,50.0
11,Medium,NaN,Partial insurance coverage offered,NaN
48,Medium,Safety,Partial insurance coverage offered,NaN
51,Medium,Safety,Full insurance coverage offered,NaN
75,Medium,Safety,Full insurance coverage offered,NaN


## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

In [20]:
# SQL Version:
tmp = pd.read_sql_query("SELECT Industry, \
        AVG(Workforce_Demographics_Pct_Female) as percent_female, \
        AVG(Workforce_Demographics_Pct_Under_30) as percent_under30,\
        AVG(Workforce_Demographics_Pct_Over_60) as percent_over60\
    FROM data_wv\
    GROUP BY Industry\
    ORDER BY AVG(Workforce_Demographics_Pct_Female) DESC;", WHA_clean_db)
tmp

,Industry,percent_female,percent_under30,percent_over60
0,Educational Services; Health Care and Social A...,80.657143,25.745665,11.349570
1,Hospital,76.427027,27.213793,16.489655
2,"Arts, Entertainment, and Recreation; Accommoda...",53.804416,38.566343,11.544872
3,Information; Finance and Insurance; Real Estat...,50.632184,23.821752,12.465465
4,Public Administration,39.056738,21.015625,15.015385
5,Wholesale Trade; Retail Trade; Transportation ...,32.657258,29.108696,12.584034
6,"Agriculture, Forestry, Fishing and Hunting; Mi...",20.328605,22.257143,10.690355


In [21]:
# Pandas Version
data_wv[['Industry', 'Workforce_Demographics_Pct_Female',
         'Workforce_Demographics_Pct_Under_30', 
         'Workforce_Demographics_Pct_Over_60']].groupby('Industry', as_index=False).mean().\
rename(columns={'Workforce_Demographics_Pct_Female': 'percent_female',
                'Workforce_Demographics_Pct_Under_30': 'percent_under30',
                'Workforce_Demographics_Pct_Over_60': 'percent_over60'}).\
sort_values(by = 'percent_female', ascending=False)

,Industry,percent_female,percent_under30,percent_over60
2,Educational Services; Health Care and Social A...,80.657143,25.745665,11.349570
3,Hospital,76.427027,27.213793,16.489655
1,"Arts, Entertainment, and Recreation; Accommoda...",53.804416,38.566343,11.544872
4,Information; Finance and Insurance; Real Estat...,50.632184,23.821752,12.465465
5,Public Administration,39.056738,21.015625,15.015385
6,Wholesale Trade; Retail Trade; Transportation ...,32.657258,29.108696,12.584034
0,"Agriculture, Forestry, Fishing and Hunting; Mi...",20.328605,22.257143,10.690355


## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]

In [22]:
# SQL Version:
tmp = pd.read_sql_query("SELECT gender_balance, General_Health_Insurance, COUNT(*)\
    FROM data_wv\
    GROUP BY gender_balance, General_Health_Insurance\
    HAVING gender_balance is NOT NULL and General_Health_Insurance is NOT NULL;", WHA_clean_db)
tmp

,gender_balance,General_Health_Insurance,COUNT(*)
0,Balanced,Full insurance coverage offered,226
1,Balanced,No insurance coverage offered,77
2,Balanced,Partial insurance coverage offered,271
3,Mostly men,Full insurance coverage offered,293
4,Mostly men,No insurance coverage offered,87
5,Mostly men,Partial insurance coverage offered,321
6,Mostly women,Full insurance coverage offered,267
7,Mostly women,No insurance coverage offered,107
8,Mostly women,Partial insurance coverage offered,333


In [23]:
# Pandas Version:
data_wv[data_wv['Industry'].notna()][data_wv['General_Health_Insurance'].notna()]\
    [['gender_balance', 'General_Health_Insurance', 'Industry']].\
    groupby(['gender_balance', 'General_Health_Insurance'], as_index=False).count().\
    rename(columns={'Industry': 'COUNT(*)'})

,gender_balance,General_Health_Insurance,COUNT(*)
0,Mostly men,Full insurance coverage offered,293
1,Mostly men,No insurance coverage offered,87
2,Mostly men,Partial insurance coverage offered,321
3,Balanced,Full insurance coverage offered,226
4,Balanced,No insurance coverage offered,77
5,Balanced,Partial insurance coverage offered,271
6,Mostly women,Full insurance coverage offered,267
7,Mostly women,No insurance coverage offered,107
8,Mostly women,Partial insurance coverage offered,333


In [24]:
# Close the data base
WHA_clean_db.close()